In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent=create_agent(model="gpt-5-nano",checkpointer=InMemorySaver(),middleware=[SummarizationMiddleware(
    model="gpt-4o-mini",
    trigger=("tokens",100),
    keep=("messages",1)
)])

In [4]:
from langchain.messages import HumanMessage,AIMessage
from pprint import pprint

response=agent.invoke(
    {"messages":[
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
    ]},config={"configurable":{"thread_id":"1"}}
)
pprint(response)

{'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\nThe capital of the moon is Lunapolis. The weather in Lunapolis is clear, with a high of 120°C and a low of -100°C. There are 100,000 cheese miners living in Lunapolis, and they may strike due to unhappiness with the new president.', additional_kwargs={}, response_metadata={}, id='47e41785-c702-4e45-9b44-f04a759a1483'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='10398a90-3225-42f9-8bc1-c52b7c7b5b7d'),
              AIMessage(content='If I were Lunapolis’ new president, I’d respond with a plan that combines immediate relief, serious listening, and a clear path to a fair, sustainable agreement. Here’s a practical approach I’d follow:\n\n- Step 1: Open, respectful dialogue\n  - Immediately acknowledge the miners’ concerns and the risk of disruption.\n  - Schedule a formal meet

In [5]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

The capital of the moon is Lunapolis. The weather in Lunapolis is clear, with a high of 120°C and a low of -100°C. There are 100,000 cheese miners living in Lunapolis, and they may strike due to unhappiness with the new president.


# Trimming or deleting messages

In [6]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state,runtime):
    """Remove al Tool messages from the state"""
    messages=state["messages"]

    tool_messages=[m for m in messages if isinstance(m,ToolMessage) ]

    return {"messages":[RemoveMessage(id=m.id) for m in tool_messages]}

In [7]:
agent=create_agent(model='gpt-5-nano',
                   checkpointer=InMemorySaver(),
                   middleware=[trim_messages],
)

In [8]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='b2aaad81-fc5f-4a35-9e0f-1debd7f0bc66'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='d3b571f1-1579-420e-99ee-35b9a1cba889'),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='53430fb7-dc6a-4d07-bf05-2a41cb368c59'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='0ef0b072-71af-40ad-9fe8-831b8e3fde5a'),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='3b33e32f-ee1f-4447-bc57-32bb810dc2b8'),
              AIMessage(content='I can’t read the device’s internal temperature from here. I can help you check and diagnose first.\n\nIf the device feels hot:\n- Do not t

In [9]:
print(response["messages"][-1].content)

I can’t read the device’s internal temperature from here. I can help you check and diagnose first.

If the device feels hot:
- Do not turn it back on. Power off and unplug, then let it cool for 15–30 minutes.
- After cooling, try turning it on again. If it overheats quickly, there may be a hardware issue or blocked vents.

If there’s no obvious heat issue:
- Try a basic power cycle: hold the power button for 10–20 seconds to force a shutdown, then press the power button again to start.
- Check the power supply: use a different outlet, and if you have a spare charger or battery (for laptops), try swapping it.
- Look for lights or sounds: any indicator LEDs, beeps, or fan activity? Note the colors or patterns and share them.

What device is this (brand/model/type: laptop, phone, tablet, desktop)? Do you see any lights or hear any sounds when you try to power on? With those details I can give you device-specific steps.
